# 分类文本词性标注
词性标注(`Part-of-Speech Tagging`):训练一个分类器来算出哪个后缀最有信息量。首先，让我们找出最常见的后缀。

In [3]:
import nltk
from nltk.corpus import brown

suffix_fdist = nltk.FreqDist()
for word in brown.words():
    word = word.lower()
    suffix_fdist[word[-1:]] += 1
    suffix_fdist[word[-2:]] += 1
    suffix_fdist[word[-3:]] += 1

common_suffixes = suffix_fdist.keys()[:100]
common_suffixes

[u'vre',
 u'g/l',
 u'mny',
 u'.32',
 u"k's",
 u'cwt',
 u'cth',
 u'*yt',
 u'ajk',
 u'pce',
 u"y'n",
 u'sch',
 u'.e.',
 u'cm.',
 u'xts',
 u'aud',
 u'rek',
 u'aui',
 u'/3%',
 u'oeb',
 u'pth',
 u'aum',
 u'aul',
 u'xty',
 u'aun',
 u'sce',
 u'aus',
 u'aur',
 u'aut',
 u'343',
 u'-ho',
 u'aux',
 u'$40',
 u'348',
 u'yms',
 u'$45',
 u'xth',
 u'mns',
 u'sca',
 u'hce',
 u'.k.',
 u'*yr',
 u'606',
 u'fur',
 u'nw.',
 u'ala',
 u'lbs',
 u'aix',
 u'upi',
 u'29%',
 u'fha',
 u'upa',
 u'nmr',
 u'298',
 u'upy',
 u'297',
 u'upt',
 u'ups',
 u'290',
 u'bl',
 u'27%',
 u'*yp',
 u'ab/',
 u'270',
 u'271',
 u'ucy',
 u'273',
 u'274',
 u'275',
 u'276',
 u'3a',
 u'7th',
 u'air',
 u'ptu',
 u'nen',
 u'ghn',
 u'nel',
 u'zur',
 u'nek',
 u'oze',
 u'nei',
 u'ned',
 u'nee',
 u'kus',
 u'nec',
 u'kup',
 u'jac',
 u'ain',
 u'nez',
 u'ak.',
 u'nex',
 u'ney',
 u'new',
 u'net',
 u'neu',
 u'ner',
 u'nes',
 u'mee',
 u'med',
 u'meg']

定义一个特征提取函数，检查给定单词的后缀。

In [4]:
def pos_features(word):
    features = {}
    for suffix in common_suffixes:
        features['endswith(%s)' % suffix] = word.lower().endswith(suffix)
    return features

定义特征提取器，用它来训练新的“[决策树](https://www.safaribooksonline.com/library/view/natural-language-processing/9780596803346/ch06s04.html)”的分类器。

In [5]:
tagged_words = brown.tagged_words(categories='news')
featuresets = [(pos_features(n), g) for (n,g) in tagged_words]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.DecisionTreeClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.14639482844356042

In [6]:
classifier.classify(pos_features('cats'))

u'NN'

决策树模型的优点是容易解释。可以指示NLTK将它们以伪代码形式输出。

```py
>>> print classifier.pseudocode(depth=4)
if endswith(,) == True: return ','
if endswith(,) == False:
  if endswith(the) == True: return 'AT'
  if endswith(the) == False:
    if endswith(s) == True:
      if endswith(is) == True: return 'BEZ'
      if endswith(is) == False: return 'VBZ'
    if endswith(s) == False:
      if endswith(.) == True: return '.'
      if endswith(.) == False: return 'NN'
```

### 探索上下文语境
语境特征往往提供关于正确标记的强大线索——例如：标注词fly时，如果知道它前面的词是“a”，能够确定它是名词，而不是动词。

In [7]:
def pos_features(sentence, i):
    features = {"suffix(1)": sentence[i][-1:],
                "suffix(2)": sentence[i][-2:],
                "suffix(3)": sentence[i][-3:]}
    if i == 0:
        features["prev-word"] = "<START>"
    else:
        features["prev-word"] = sentence[i-1]
    return features

pos_features(brown.sents()[0], 8)

{'prev-word': u'an',
 'suffix(1)': u'n',
 'suffix(2)': u'on',
 'suffix(3)': u'ion'}

In [8]:
tagged_sents = brown.tagged_sents(categories='news')
featuresets = []
for tagged_sent in tagged_sents:
    untagged_sent = nltk.tag.untag(tagged_sent)
    for i, (word, tag) in enumerate(tagged_sent):
        featuresets.append((pos_features(untagged_sent, i), tag))

size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.7891596220785678

### 序列分类
为了获取相关分类任务之间的依赖关系，我们可以使用`joint classifier`模型，为一些相关的输入选择适当的标签。在词性标注的例子中，可以使用各种不同的`sequence classifier`模型为给定的句子中的所有词选择词性标签。

一种称为`consecutive classification`或`greedy sequence classification`的序列分类器策略，为第一个输入找到最有可能的类标签，然后在此基础上找到下一个输入的最佳的标签。这个过程可以不断重复直到所有的输入都被贴上标签。

使用连续分类器进行词性标注。

In [9]:
def pos_features(sentence, i, history):
    features = {"suffix(1)": sentence[i][-1:],
                "suffix(2)": sentence[i][-2:],
                "suffix(3)": sentence[i][-3:]}
    if i == 0:
        features["prev-word"] = "<START>"
        features["prev-tag"] = "<START>"
    else:
        features["prev-word"] = sentence[i-1]
        features["prev-tag"] = history[i-1]
    return features

class ConsecutivePosTagger(nltk.TaggerI):
    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = pos_features(untagged_sent, i, history)
                train_set.append( (featureset, tag) )
                history.append(tag)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)
    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = pos_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

tagged_sents = brown.tagged_sents(categories='news')
size = int(len(tagged_sents) * 0.1)
train_sents, test_sents = tagged_sents[size:], tagged_sents[:size]
tagger = ConsecutivePosTagger(train_sents)
tagger.evaluate(test_sents)

0.7980528511821975

### 其他序列分类方法
这种方法的缺点是一旦做出决定便无法更改。例如：如果决定将一个词标注为名词，但后来发现应该是动词，那也没有办法修复我们的错误了。解决这个问题的方法是采取转型策略(`transformational strategy`)。转型联合分类(`Transformational joint classifiers`)的工作原理是为输入的标签创建一个初始值，然后反复提炼该值，尝试修复相关输入之间的不一致。Brill标注器，是使用这种策略的。

另一种方案是为词性标记所有可能的序列打分，选择总得得分最高的序列。隐马尔可夫模型(`Hidden Markov Models`)采取了这种方法。隐马尔可夫模型类似于连续分类器，不光考虑输入也考虑已预测标记的历史。然而，不是简单地找出一个给定词的单个最好标签，而是为标记产生一个概率分布。然后将这些概率结合起来计算标记序列的概率得分，最后选择最高概率的标记序列。不过，可能的标签序列数量相当大。给定拥有30个标签的标记集，大约有600万亿(30^10)种方式来标记一个10个词的句子。为了避免单独考虑所有这些可能的序列，隐马尔可夫模型要求特征提取器只考虑最近的标记来有效地找出最有可能的标记序列。特别是，对每个连续的词索引i，当前的及以前的每个可能的标记都将计算得分。这种基础的方法被两个更先进的模型所采用，它们被称为最大熵隐马尔可夫模型(`Maximum Entropy Markov Models`)和线性链条件随机场模型(`Linear-Chain Conditional Random Field Models`)。
